In [ ]:
!pip install transformers
!pip install seqeval
!pip install tensorboardx
!pip install torchvision==0.4.0 -f https://download.pytorch.org/whl/cu100/torch_stable.html --no-cache-dir
!rm -rf .git
!rm -rf apex
!git clone https://github.com/NVIDIA/apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex
!pip3 install simpletransformers
!pip install Afinn

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
cd /content/drive/My\ Drive/Colab\ Notebooks

In [ ]:
import numpy as np
import pandas as pd
import torch
import gc
import csv

from CsvReader import ReadCsv
from HelperFunctions import ShuffleData, replace_all, ReplaceSentimentsWithIndexes, SplitArticlesIntoSentiments, getMetrics, TransformDataIntoDataframe, CalculateWeights
from TrainEvalModel import TrainModelForMultiClass, EvalFromModel

data, categories, tendencies = ReadCsv('/content/drive/My Drive/Colab Notebooks Pascal/bert-etiki/etiki-data','test.csv','companies.csv', 'categories.csv','references.csv','tendencies.csv', 'topics.csv')
sentimentData = ReplaceSentimentsWithIndexes(data[:,[0,7,13]])
labels = np.unique(sentimentData[:,1])
weights = CalculateWeights(labels,sentimentData)

for i in range(1,5):
  for j in range(2,6):
    positiveArticles, negativeArticles, controversialArticles = SplitArticlesIntoSentiments(sentimentData)
    positiveArticles = ShuffleData(positiveArticles)
    negativeArticles = ShuffleData(negativeArticles)
    controversialArticles = ShuffleData(controversialArticles)

    trainData = np.vstack((positiveArticles[:int(len(positiveArticles)*0.7)],controversialArticles[:int(len(controversialArticles)*0.7)],negativeArticles[:int(len(negativeArticles)*0.7)]))
    testData = np.vstack((positiveArticles[int(len(positiveArticles)*0.7):],controversialArticles[int(len(controversialArticles)*0.7):],negativeArticles[int(len(negativeArticles)*0.7):]))
    train_df = TransformDataIntoDataframe(trainData)
    eval_df = TransformDataIntoDataframe(testData)
#------------------------------XLNet------------------------------
    algo = 'xlnet'        
    args = {'reprocess_input_data': True,
              'overwrite_output_dir': True,
              'num_train_epochs':j ,
              'silent':True,
              "gradient_accumulation_steps": i,
              }

    model = TrainModelForMultiClass(algo, 'xlnet-base-cased',train_df,3,args)
    result, model_outputs, wrong_predictions = EvalFromModel(model,eval_df)

    with open('results/gradient/'+str(j)+'epoch/'+algo+'-'+i'-mcc.csv', 'a', newline='') as f:
      writer = csv.writer(f)
      writer.writerow([result['mcc'],result["acc"],result["eval_loss"]])

    getMetrics(result["matr"],3,'gradient/'+str(j)+'epoch/'+algo+'-'+i)